---

## Data Analysis

- This file differs from [2_data_analysis_1_base_data.ipynb](2_data_analysis_1_base_data.ipynb) in that it:
    - scales the base cleaned data created in [1_data_cleaning.ipynb](1_data_cleaning.ipynb).

Source dataset: 247076 rows × 37 columns
Processed and analyzed dataset: 247076 rows × 37 columns

---

In [16]:
# package imports go here
import pandas as pd
import numpy as np
import fastparquet as fp
import os
import sys
import pickle
import matplotlib.pyplot as plt
import importlib

sys.path.insert(1, 'pkgs')
import ml_analysis as mlanlys
import ml_clean_feature as mlclean

import warnings
warnings.filterwarnings("ignore")

---

## 1. Read the cleaned dataset from file

---

In [17]:
# Path to results
year = 2015
source_path     = "data/"
clean_file      = source_path + 'brfss_' + str(year) + '_clean.parquet.gzip'

report_path = 'reports/'
performance_report = report_path + 'performance_report.pkl'

# BE SURE TO UPDATE THE LABEL FOR THIS ANALYSIS
dataset_label = 'RandomUndersampled Dataset'

file_label = dataset_label.lower().replace(' ','_')

detailed_performance_report = report_path + file_label + '_detailed_performance_report.txt'

In [18]:
# Read final cleaned dataset from parquet file
df = pd.read_parquet(clean_file, engine="fastparquet")

In [19]:
diabetes_labels = df.columns

In [20]:
df.shape

(253680, 22)

---

## 2. Prepare the dataset for analysis

- Split the dataset into features and labels.
- Split the dataset into training and testing sets.
- Scale the dataset

---

In [21]:
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [22]:
# reload any changes to mlanlys
importlib.reload(mlanlys)

target = 'diabetes'
# Dictionary defining modification to be made to the base dataset
operation_dict = {  'target_column'     :  target,
                    'convert_to_binary' :  True,
                    'scaler'            : 'standard', # options: none, standard, minmax
                    'random_sample'     : 'undersample'      # options: none, undersample, oversample
                    }

# This insures that df if not modified during the call to modify_base_dataset()
df_modified = df.copy()

# Modify the base dataset
# data is returned where: X_train, X_test, y_train, y_test = data
data = mlanlys.modify_base_dataset(df_modified, operation_dict)

Base Dataset Modifications in Process
-------------------------------------
**Operation:target_column  diabetes
**Operation:convert_to_binary  True
  -- Converting dataset to binary (0,1) from (0,1,2)


****Cleaning Feature: diabetes
  Initial Unique features in [diabetes]:  [0. 1. 2.]
  values_to_drop: ********* NO Parameters were specified *********
  translate: {1: 0, 2: 1}
  scale: ********* NO Parameters were specified *********
  FINAL Unique features in [diabetes]:  [0. 1.]
**Operation:scaler  standard
  -- Performing train_test_split on dataframe with target:'diabetes'
     -- Run automatically before scalar or random_sample operations
  -- Performing StandardScaler on X_train: Updates X_train, y_test
**Operation:random_sample  undersample
  -- Performing RandomUnderSampler on X_train, y_train: Updates X_train, y_train

Dataframe, Train Test Summary
-----------------------------
Dataframe: (253680, 22)  Data:4, X_train:190260, y_train:190260, X_test:63420, y_test:63420
ValueCou

In [23]:
# Print some statistics about the original df and the modified dataframe
print(f"Original Dataframe")
print(f"------------------")
print(f"df.shape: {df.shape}")
print(f"df[{target}].value_counts:  {df[target].value_counts()}")

print(f"\nModified Dataframe")
print(f"------------------")
print(f"df_modified.shape: {df_modified.shape}")
print(f"df_modified[{target}].value_counts:  {df_modified[target].value_counts()}")

Original Dataframe
------------------
df.shape: (253680, 22)
df[diabetes].value_counts:  diabetes
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

Modified Dataframe
------------------
df_modified.shape: (253680, 22)
df_modified[diabetes].value_counts:  diabetes
0.0    218334
1.0     35346
Name: count, dtype: int64


In [24]:
X_train, X_test, y_train, y_test = data
print(f"Dataframe: {df_modified.shape}  Data:{len(data)}, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")

Dataframe: (253680, 22)  Data:4, X_train:52762, y_train:52762, X_test:63420, y_test:63420


In [25]:
y_train.value_counts()

diabetes
0.0    26381
1.0    26381
Name: count, dtype: int64

In [26]:
y_test.value_counts()

diabetes
0.0    54455
1.0     8965
Name: count, dtype: int64

---

## 3. Optimization

---

#### 3.1 Pre-optimization metric results

- The summary report of the metrics for all pre-optimization runs is here:  [performance_report.txt](reports/performance_report.txt)
- The details of the runs are contained in these file:
    - [base_dataset_detailed_performance_report.txt](reports/base_dataset_detailed_performance_report.txt)
    - [binary_dataset_detailed_performance_report.txt](reports/binary_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [randomoversample_dataset_detailed_performance_report.txt](reports/randomoversample_dataset_detailed_performance.txt)
    - [cluster_dataset_detailed_performance_report.txt](reports/cluster_dataset_detailed_performance_report.txt)
    - [randomundersampled_dataset_detailed_performance_report.txt](reports/randomundersampled_dataset_detailed_performance_report.txt)
    - [minmax_scaled_dataset_detailed_performance_report.txt](reports/minmax_scaled_dataset_detailed_performance_report.txt)
    - [smoteen_dataset_detailed_performance_report.txt](reports/smoteen_dataset_detailed_performance_report.txt)
    - [standard_scaled_dataset_detailed_performance_report.txt](reports/standard_scaled_dataset_detailed_performance_report.txt)
    - [smote_dataset_detailed_performance_report.txt](reports/smote_dataset_detailed_performance_report.txt)

#### 3.2 Optimization Dataset used

**Note:**  Modify the dataset as desired in Section 2.
<br><br>
Currently the dataset uses the Base dataset for 2015 with the following modifications:
- **Target converted to Binary**:  (0,1) from (0,1,2).
    - Base:  0: No diabetes, 1: Pre-diabetes, 2: have diabetes
    - binary: 0: No diabetes/pre-diabetes, 1: have diabetes
- Scaled the data with **StandardScaler**
- Resampled the data with **RandomUnderSampler**


#### 3.2 Optimization Analysis

In [27]:
# Note:  Modify the dataset as desired in Section 2.
# Currently the dataset uses the Base dataset for 2015 with the following modifications

In [28]:
# Dataset summary:
X_train, X_test, y_train, y_test = data
print(f"Dataset Lens, X_train:{len(X_train)}, y_train:{len(y_train)}, X_test:{len(X_test)}, y_test:{len(y_test)}")
print(f"y_train value_counts: {y_train.value_counts()}")
print(f"y_test value_counts: {y_test.value_counts()}")


Dataset Lens, X_train:52762, y_train:52762, X_test:63420, y_test:63420
y_train value_counts: diabetes
0.0    26381
1.0    26381
Name: count, dtype: int64
y_test value_counts: diabetes
0.0    54455
1.0     8965
Name: count, dtype: int64


##### 3.2.1 Perform Optimization:

- Datamodel:  2015 Diabetes Data Set
- Optimization approaches: DecisionTreeRegressor RandomizedSearchCV


In [29]:
df_modified_sample = df_modified.head(100)

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the parameter grid
param_distributions = {
    'max_depth': [int(x) for x in np.linspace(10, 110, num=11)],  # Depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2']  # Number of features to consider for the best split
}

# Initialize the Decision Tree Regressor
# dtr = DecisionTreeRegressor()
dtr = DecisionTreeClassifier()

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    dtr,
    param_distributions=param_distributions,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=5,        # 5-fold cross-validation
    verbose=1,   # Verbosity mode
    n_jobs=-1,   # Use all available cores
    random_state=42  # Seed for reproducibility
)

# Fit the model
random_search.fit(X_train, y_train)

# Best model
best_dtr = random_search.best_estimator_

# Make predictions on the test set
y_pred_dtr = best_dtr.predict(X_test)

# Evaluate the regressor
mse_dtr = mean_squared_error(y_test, y_pred_dtr)
print(f"Best Parameters for Decision Tree Regressor: {random_search.best_params_}")
print(f"Mean Squared Error of Decision Tree Regressor: {mse_dtr}")


Fitting 5 folds for each of 100 candidates, totalling 500 fits


/Users/eliaporter/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/eliaporter/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/eliaporter/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0'`.
  warnings.warn(
/Users/eliaporter/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/tree/_classes.py:277: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To ke

Best Parameters for Decision Tree Regressor: {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 10}
Mean Squared Error of Decision Tree Regressor: 0.1900281638899133


---

## 4. Conclusions

---

---
---
### Notes: Examples
---
---

Starter Code Below.  Delete once 3.2.1 is completed

- See Class example:  **module 14, day 3, activity 2**: hypoerparameters_solution.ipynb
- See Class example:  **module 14, day 3, activity 5**: fourth_model_solution.ipynb


---

Example 1

---

---

Example 2

---

In [30]:
# Example 2
# Create the grid search estimator along with a parameter object containing the values to adjust.
# Try adjusting n_neighbors with values of 1 through 19. Adjust leaf_size by using 10, 50, 100, and 500.
# Include both uniform and distance options for weights.
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
    'weights': ['uniform', 'distance'],
    'leaf_size': [10, 50, 100, 500]
}
grid_clf = GridSearchCV(grid_tuned_model, param_grid, verbose=3)

NameError: name 'grid_tuned_model' is not defined

In [ ]:
# Fit the model by using the grid search estimator.
# This will take the KNN model and try each combination of parameters.
grid_clf.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid_clf.best_params_)

In [ ]:
# Print the classification report for the best model
grid_y_pred = grid_clf.predict(X_test)
print(classification_report(y_test, grid_y_pred,
                            target_names=target_names))

---

Example 3

---

In [ ]:
# Example 3
# Create the parameter object for the randomized search estimator.
# Try adjusting n_neighbors with values of 1 through 19. 
# Adjust leaf_size by using a range from 1 to 500.
# Include both uniform and distance options for weights.
param_grid = {
    'n_neighbors': np.arange(1,20,2),
    'weights': ['uniform', 'distance'],
    'leaf_size': np.arange(1, 500)
}
param_grid

In [ ]:
# Create the randomized search estimator
from sklearn.model_selection import RandomizedSearchCV
random_clf = RandomizedSearchCV(random_tuned_model, param_grid, random_state=0, verbose=3)

In [ ]:
# Fit the model by using the randomized search estimator.
random_clf.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset
print(random_clf.best_params_)

In [ ]:
# Make predictions with the hypertuned model
random_tuned_pred = random_clf.predict(X_test)

In [ ]:
# Calculate the classification report
print(classification_report(y_test, random_tuned_pred,
                            target_names=target_names))

---

Example 3

---

E3.1 Compute max_depth

In [ ]:
# Example 1
models = {'train_score': [], 'test_score': [], 'max_depth': []}

for depth in range(1,10):
    models['max_depth'].append(depth)
    model = RandomForestClassifier(n_estimators=100, max_depth=depth)
    model.fit(X_train_pca, y_train_encoded)
    y_test_pred = model.predict(X_test_pca)
    y_train_pred = model.predict(X_train_pca)

    models['train_score'].append(balanced_accuracy_score(y_train_encoded, y_train_pred))
    models['test_score'].append(balanced_accuracy_score(y_test_encoded, y_test_pred))

models_df = pd.DataFrame(models)

In [ ]:
models_df.plot(x='max_depth')

E3.2 Compute n_estimators

In [ ]:
models = {'train_score': [], 'test_score': [], 'n_estimators': []}

for n in [50, 100, 500, 1000]:
    models['n_estimators'].append(n)
    model = RandomForestClassifier(n_estimators=n, max_depth=7)
    model.fit(X_train_pca, y_train_encoded)
    y_test_pred = model.predict(X_test_pca)
    y_train_pred = model.predict(X_train_pca)

    models['train_score'].append(balanced_accuracy_score(y_train_encoded, y_train_pred))
    models['test_score'].append(balanced_accuracy_score(y_test_encoded, y_test_pred))

models_df = pd.DataFrame(models)

In [ ]:
models_df.plot(x='n_estimators')

In [ ]:
# Define best depth and estimators
mdepth = 7
nestimators = 1000

model = RandomForestClassifier(n_estimators=1000, max_depth=7)
model.fit(X_train_pca, y_train_encoded)
y_test_pred = model.predict(X_test_pca)
y_train_pred = model.predict(X_train_pca)

print(f"Train balanced accuracy: {balanced_accuracy_score(y_train_encoded, y_train_pred)}")
print(f"Test balanced accuracy: {balanced_accuracy_score(y_test_encoded, y_test_pred)}")

# print classificaiton report